<a href="https://colab.research.google.com/github/crazat/crazat.githurb.io/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%84%B1%EB%8A%A5%ED%96%A5%EC%83%81_%EC%8B%A4%EC%8A%B5(AutoML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 타이타닉 생존자 데이터셋 다운로드 받아 구글드라이브에 올리기
    - 학습세트: https://drive.google.com/file/d/17DUdpAGVIPJ5_Cghku38YG5ugN6-SiNX/view?usp=sharing
    - 평가세트: https://drive.google.com/file/d/1ZhrzzI81mN_rFWXw68eO2a2NQ8vg8IlZ/view?usp=sharing
    - 평가세트 정답: https://drive.google.com/file/d/1kBJwGeAI1y81-gL8AAVVs4ealPHTcNNS/view?usp=sharing

- 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [ ]:
DATA_PATH = "/content/drive/MyDrive/머신러닝/data/"
DATA_PATH

'/content/drive/MyDrive/머신러닝/data/'

- 시드값

In [ ]:
SEED = 42

- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터
test_target = pd.read_csv(f"{DATA_PATH}titanic_test_target.csv") # 테스트데이터 정답값
train.shape , test.shape , test_target.shape

((916, 12), (393, 11), (393, 2))

- 결측치 확인하기

In [ ]:
train.isnull().sum()

,0
passengerid,0
survived,0
pclass,0
name,0
gender,0
age,180
sibsp,0
parch,0
ticket,0
fare,0


In [ ]:
test.isnull().sum()

,0
passengerid,0
pclass,0
name,0
gender,0
age,83
sibsp,0
parch,0
ticket,0
fare,1
cabin,308


- 학습 데이터에서 얻은 통계량을 이용하여 결측치를 채워야한다.

In [ ]:
age_mean = train["age"].mean() # 평균
fare_median = train["fare"].median() # 중앙값
cabin_unk = "UNK" # 새로운 범주
embarked_mode = train["embarked"].mode()[0] # 최빈값
age_mean , fare_median ,cabin_unk , embarked_mode

(29.904891304347824, 14.5, 'UNK', 'S')

- 학습데이터 결측치 처리

In [ ]:
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)

- 테스트데이터 결측치 처리

In [ ]:
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

In [ ]:
train.isnull().sum().sum() , test.isnull().sum().sum()

(0, 0)

- 특성으로 사용할 변수 추가하기

In [ ]:
cols = ["age","sibsp","parch","fare","pclass","gender","embarked","cabin"]
train_ft = train[cols].copy()
test_ft = test[cols].copy() # 테스트 데이터
train_ft.shape, test_ft.shape

((916, 8), (393, 8))

- category_encoders 설치하기

In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 831.6 kB/s eta 0:00:00


- import

In [ ]:
import category_encoders as ce

In [ ]:
train_ft.columns

Index(['age', 'sibsp', 'parch', 'fare', 'pclass', 'gender', 'embarked',
       'cabin'],
      dtype='object')

- One- Hot Encoding


In [ ]:
cols = ['gender', 'embarked']

enc = ce.one_hot.OneHotEncoder(return_df=True)

# 학습데이터
tmp = enc.fit_transform(train_ft[cols]).add_prefix("onehot_")
train_ft = pd.concat([train_ft,tmp],axis=1)

# 테스트데이터
tmp = enc.transform(test_ft[cols]).add_prefix("onehot_")
test_ft = pd.concat([test_ft,tmp],axis=1)

- Count Encoding

In [ ]:
cols = ['cabin']

enc = ce.count.CountEncoder(return_df=True,handle_unknown=0)

# 학습데이터
tmp = enc.fit_transform(train_ft[cols]).add_prefix("count_")
train_ft = pd.concat([train_ft,tmp],axis=1)

# 테스트데이터
tmp = enc.transform(test_ft[cols]).add_prefix("count_")
test_ft = pd.concat([test_ft,tmp],axis=1)

- 범주형 컬럼 제거

In [ ]:
cols = ['gender', 'embarked','cabin']
train_ft = train_ft.drop(columns = cols)
test_ft = test_ft.drop(columns = cols)

train_ft.shape , test_ft.shape

((916, 11), (393, 11))

- 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# 학습데이터
train_ft = pd.DataFrame(
    scaler.fit_transform(train_ft),
    columns = train_ft.columns
)

# 테스트 데이터
test_ft = pd.DataFrame(
    scaler.transform(test_ft),
    columns = test_ft.columns
)

- 타겟값 target 변수에 저장

In [ ]:
target = train["survived"]
target

,survived
0,0
1,0
2,0
3,1
4,1
...,...
911,1
912,0
913,0
914,0


# AutoML(Automated machine learning)
- 시간 소모적이고 반복적인 기계 학습 모델 개발 작업을 자동화하는 프로세스
- 데이터 과학자, 분석가 및 개발자는 모델 품질을 유지하면서 확장성, 효율성 및 생산성이 높은 ML 모델을 빌드할 수 있다.

## FLAML(A Fast Library for Automated Machine Learning & Tuning)
- 머신러닝 학습 및 하이퍼파라미터 튜닝을 자동화해 주는 라이브러리
- https://microsoft.github.io/FLAML/docs/Use-Cases/Task-Oriented-AutoML

- flaml 설치하기

In [ ]:
!pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 4.3 MB/s eta 0:00:00


- `AutoML` 클래스의 `fit` 메서드 주요 파라미터
    - metric
        - 평가지표
        - ex) 'roc_auc'
    - task
        - 작업 유형
        - ex) 'classification'
    - estimator_list
        - FLAML에서 제공해주는 모델들의 별칭을 리스트에 넣어주면 된다.
        - 생략시 자동으로 모델들이 선택된다.
        - ex) ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
    - time_budget
        - 수행 시간
    - ensemble
        - 튜닝 후 스태킹 앙상블 여부
        - False(기본값) : 최상의 모델을 선택해서 학습
        - True: 스태킹하여 앙상블
        - `dict` 예시
            - {'final_estimator' : 사이킷런 모델 객체 }
    - n_splits
        - 교차검증 폴드 수(기본 5)
    - seed
        - 시드값
    - early_stop
        - True or False(기본값)
        - 튜닝이 수렴할 경우 조기 중지여부

In [ ]:
from flaml import AutoML

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
auto_ml = AutoML()
params = {
    "metric": "roc_auc",
    "task": "classification",
    "time_budget": 60, # 초단위
    "seed": SEED,
    "early_stop":True
}
auto_ml.fit(train_ft, target, **params)

[flaml.automl.logger: 02-26 09:33:45] {1728} INFO - task = classification
[flaml.automl.logger: 02-26 09:33:45] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 02-26 09:33:45] {1838} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 02-26 09:33:45] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 02-26 09:33:45] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 02-26 09:33:46] {2393} INFO - Estimated sufficient time budget=11052s. Estimated necessary time budget=256s.
[flaml.automl.logger: 02-26 09:33:46] {2442} INFO -  at 1.2s,	estimator lgbm's best error=0.1018,	best estimator lgbm's best error=0.1018
[flaml.automl.logger: 02-26 09:33:46] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 02-26 09:33:47] {2442} INFO -  at 2.2s,	estimator lgbm's best error=0.1018,	best estimator lgbm's best error=0.1018
[flaml.automl.logger: 02-2

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 02-26 09:33:49] {2442} INFO -  at 4.3s,	estimator sgd's best error=0.1113,	best estimator lgbm's best error=0.1018
[flaml.automl.logger: 02-26 09:33:49] {2258} INFO - iteration 4, current learner lgbm
[flaml.automl.logger: 02-26 09:33:50] {2442} INFO -  at 5.0s,	estimator lgbm's best error=0.0989,	best estimator lgbm's best error=0.0989
[flaml.automl.logger: 02-26 09:33:50] {2258} INFO - iteration 5, current learner sgd
[flaml.automl.logger: 02-26 09:33:50] {2442} INFO -  at 5.6s,	estimator sgd's best error=0.1113,	best estimator lgbm's best error=0.0989
[flaml.automl.logger: 02-26 09:33:50] {2258} INFO - iteration 6, current learner xgboost
[flaml.automl.logger: 02-26 09:33:52] {2442} INFO -  at 6.7s,	estimator xgboost's best error=0.0987,	best estimator xgboost's best error=0.0987
[flaml.automl.logger: 02-26 09:33:52] {2258} INFO - iteration 7, current learner lgbm
[flaml.automl.logger: 02-26 09:33:52] {2442} INFO -  at 6.9s,	estimator lgbm's best error=0.0963,	

- 선택된 모델

In [ ]:
auto_ml.model.estimator

LGBMClassifier(colsample_bytree=0.9253539584169304,
               learning_rate=0.06981033920507693, max_bin=63,
               min_child_samples=4, n_estimators=21, n_jobs=-1, num_leaves=14,
               reg_alpha=0.0013176806294550453, reg_lambda=10.23225524421718,
               verbose=-1)

- 튜닝된 하이퍼파라미터

In [ ]:
auto_ml.best_config

{'n_estimators': 21,
 'num_leaves': 14,
 'min_child_samples': 4,
 'learning_rate': 0.06981033920507693,
 'log_max_bin': 6,
 'colsample_bytree': 0.9253539584169304,
 'reg_alpha': 0.0013176806294550453,
 'reg_lambda': 10.23225524421718}

- 선택된 모델 교차검증 best 점수

In [ ]:
auto_ml.best_loss

0.08851676292179725

In [ ]:
1 - auto_ml.best_loss

0.9114832370782028

- 테스트 데이터 예측및 평가

In [ ]:
from sklearn.metrics import roc_auc_score
y_test = test_target["survived"] # 테스트셋 y값
pred = auto_ml.predict_proba(test_ft)[:,1] # 예측
roc_auc_score(y_test,pred) # AUC 평가

0.8886238279095422

- ensemble 해보기

In [ ]:
auto_ml = AutoML()
params = {
    "metric": "roc_auc",
    "task": "classification",
    "time_budget": 60, # 초단위
    "seed": SEED,
    "ensemble": True,
    "early_stop":True
}
auto_ml.fit(train_ft, target, **params)

[flaml.automl.logger: 02-26 09:34:45] {1728} INFO - task = classification
[flaml.automl.logger: 02-26 09:34:45] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 02-26 09:34:45] {1838} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 02-26 09:34:45] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 02-26 09:34:45] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 02-26 09:34:45] {2393} INFO - Estimated sufficient time budget=1670s. Estimated necessary time budget=39s.
[flaml.automl.logger: 02-26 09:34:45] {2442} INFO -  at 0.2s,	estimator lgbm's best error=0.1018,	best estimator lgbm's best error=0.1018
[flaml.automl.logger: 02-26 09:34:46] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 02-26 09:34:46] {2442} INFO -  at 0.6s,	estimator lgbm's best error=0.1018,	best estimator lgbm's best error=0.1018
[flaml.automl.logger: 02-26 

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 02-26 09:34:46] {2442} INFO -  at 1.1s,	estimator sgd's best error=0.1113,	best estimator lgbm's best error=0.1018
[flaml.automl.logger: 02-26 09:34:46] {2258} INFO - iteration 3, current learner xgboost
[flaml.automl.logger: 02-26 09:34:47] {2442} INFO -  at 1.6s,	estimator xgboost's best error=0.0987,	best estimator xgboost's best error=0.0987
[flaml.automl.logger: 02-26 09:34:47] {2258} INFO - iteration 4, current learner extra_tree
[flaml.automl.logger: 02-26 09:34:47] {2442} INFO -  at 1.8s,	estimator extra_tree's best error=0.1152,	best estimator xgboost's best error=0.0987
[flaml.automl.logger: 02-26 09:34:47] {2258} INFO - iteration 5, current learner extra_tree
[flaml.automl.logger: 02-26 09:34:47] {2442} INFO -  at 2.1s,	estimator extra_tree's best error=0.1152,	best estimator xgboost's best error=0.0987
[flaml.automl.logger: 02-26 09:34:47] {2258} INFO - iteration 6, current learner lgbm
[flaml.automl.logger: 02-26 09:34:48] {2442} INFO -  at 2.3s,	esti

- 테스트 데이터 예측및 평가

In [ ]:
from sklearn.metrics import roc_auc_score
y_test = test_target["survived"] # 테스트셋 y값
pred = auto_ml.predict_proba(test_ft)[:,1] # 예측
roc_auc_score(y_test,pred) # AUC 평가

0.8947876447876447